In [3]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import mean_absolute_error
import optuna
from concurrent.futures import ProcessPoolExecutor
from joblib import Parallel, delayed
from scipy.stats import t


In [4]:
Main_data=pd.read_csv('Main_data.csv')
Schedule=pd.read_csv('Schedule.csv')
Origin_result = pd.read_csv('Origin_result.csv')
Main_data.columns

Index(['Team', 'Conference', '2PA', '3PA', 'TOV', 'FTD', 'RAA', 'PZA', 'MRA',
       'COA', 'ABA', '2SFD', '3SFD', 'NSFD', 'RAP', 'PZP', 'MRP', 'COP', 'ABP',
       'FTP', 'PACE', 'DEEPA', 'DEEPP', '2PAnd1', '3PAnd1', 'ORAP', 'OPZP',
       'OMRP', 'OCOP', 'OABP', 'ORB', 'DRB'],
      dtype='object')

In [5]:
Main_data.head()

,Team,Conference,2PA,3PA,TOV,FTD,RAA,PZA,MRA,COA,...,DEEPP,2PAnd1,3PAnd1,ORAP,OPZP,OMRP,OCOP,OABP,ORB,DRB
0,ATL,East,4492,3092,1110,1016,2221,1533,737,869,...,23.7,192,17,67.2,45.0,42.7,40.6,37.8,12.5,32.2
1,BOS,East,3914,3482,979,881,1974,1195,745,765,...,23.2,165,7,64.0,42.9,42.0,35.2,35.6,10.7,35.6
2,BKN,East,4297,3010,1076,914,2146,1387,764,822,...,19.4,173,15,65.7,39.9,43.1,41.8,36.1,11.4,32.6
3,CHA,East,4345,2788,1129,818,2164,1320,861,642,...,20.2,175,5,67.6,46.2,42.1,40.0,37.1,9.3,31.0
4,CHI,East,4709,2630,1004,935,2238,1276,1194,755,...,22.0,179,4,66.1,43.6,42.1,41.6,35.3,11.2,32.6


In [9]:
def generate_dirichlet_sample(row,multiply):
    ratios = row[['2PA', '3PA', 'DEEPA', 'FTD', 'TOV']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios  
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  
    
    dirichlet_sample = np.random.dirichlet(alpha)

    return dirichlet_sample

def generate_dirichlet_sample_2pt(row,multiply):
    ratios = row[['RAA', 'PZA', 'MRA', '2PAnd1']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios 
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  

    dirichlet_sample_2pt = np.random.dirichlet(alpha)

    return dirichlet_sample_2pt

def generate_dirichlet_sample_3pt(row,multiply):
    ratios = row[['COA', 'ABA', '3PAnd1']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios  
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  

    dirichlet_sample_3pt = np.random.dirichlet(alpha)

    return dirichlet_sample_3pt

def generate_dirichlet_sample_FT(row,multiply):
    
    ratios = row[['2SFD', '3SFD', 'NSFD']].values
    ratios = ratios.flatten()
    
    alpha_raw = ratios 
    alpha = (alpha_raw / alpha_raw.sum()) * multiply  

    dirichlet_sample_FT = np.random.dirichlet(alpha)

    return dirichlet_sample_FT

In [11]:
def generate_indicator(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return '2pt'
        elif random_value <= cumulative_probs[1]:
            return '3pt'
        elif random_value <= cumulative_probs[2]:
            return '4pt'    
        elif random_value <= cumulative_probs[3]:
            return 'FT'
        else:
            return 'TOV'
            
def generate_indicator_2pt(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return 'RA'
        elif random_value <= cumulative_probs[1]:
            return 'PZ'
        elif random_value <= cumulative_probs[2]:
            return 'MR'
        else:
            return '2PAnd1'

def generate_indicator_3pt(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return 'CO'
        elif random_value <= cumulative_probs[1]:
            return 'AB'
        else:
            return '3PAnd1'

def generate_indicator_FT(sample_dist):
        cumulative_probs = np.cumsum(sample_dist)

        random_value = np.random.uniform(0, 1)

        if random_value <= cumulative_probs[0]:
            return '2SFD'
        elif random_value <= cumulative_probs[1]:
            return '3SFD'
        else:
            return 'NSFD'

In [13]:
def simulation_function(OFF, DEF, rate, multiply, maindata):
    off_team_data = maindata[maindata['Team'] == OFF]
    def_team_data = maindata[maindata['Team'] == DEF]

    if off_team_data.empty or def_team_data.empty:
        raise ValueError("Name_error")

    sample_dist = generate_dirichlet_sample(off_team_data, multiply)
    
    indicator = generate_indicator(sample_dist)

    retry_occurred = False
    nested_result = None  # 중첩 결과 초기화

    if indicator == '2pt':
        sample_dist_step2 = generate_dirichlet_sample_2pt(off_team_data, multiply)
        indicator_step2 = generate_indicator_2pt(sample_dist_step2)
        if indicator_step2 == 'RA':
            success_prob = ((rate * off_team_data['RAP'].iloc[0]) + ((1-rate) * def_team_data['ORAP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 2
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0

        elif indicator_step2 == 'PZ':
            success_prob = ((rate * off_team_data['PZP'].iloc[0]) + ((1-rate) * def_team_data['OPZP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 2
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0

        elif indicator_step2 == 'MR':
            success_prob = ((rate * off_team_data['MRP'].iloc[0]) + ((1-rate) * def_team_data['OMRP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 2
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0

        else:
            success_prob = (off_team_data['FTP'].iloc[0] / 100)
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 3
            else:
                point = 2
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 2
                    
    elif indicator == '3pt':
        sample_dist_step2 = generate_dirichlet_sample_3pt(off_team_data, multiply)
        indicator_step2 = generate_indicator_3pt(sample_dist_step2)
        if indicator_step2 == 'CO':
            success_prob = ((rate * off_team_data['COP'].iloc[0]) + ((1-rate) * def_team_data['OCOP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 3
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0
        elif indicator_step2 == 'AB':
            success_prob = ((rate * off_team_data['ABP'].iloc[0]) + ((1-rate) * def_team_data['OABP'].iloc[0])) / 100
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 3
            else:
                point = 0
            if point == 0:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 0
            
        else:
            success_prob = (off_team_data['FTP'].iloc[0] / 100)
            random_value = np.random.uniform(0, 1)
            if random_value <= success_prob:
                point = 4
            else:
                point = 3
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = 3
    elif indicator == '4pt':
        indicator_step2 = '4pt'
        success_prob = off_team_data['DEEPP'].iloc[0] / 100  
        point = 0
        random_value = np.random.uniform(0, 1)
        if random_value <= success_prob:
            point = 4 
        else:
            point = 0
        if point == 0:
            off_orb = off_team_data['ORB'].iloc[0]
            def_drb = def_team_data['DRB'].iloc[0]
            total = off_orb + def_drb
            prob_retry = off_orb / total if total > 0 else 0
            prob_fail = def_drb / total if total > 0 else 1

            retry_prob = np.random.uniform(0, 1)
            if retry_prob <= prob_retry:
                retry_occurred = True
                nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
            else:
                point = 0

    elif indicator == 'FT':
        sample_dist_step2 = generate_dirichlet_sample_FT(off_team_data, multiply)
        indicator_step2 = generate_indicator_FT(sample_dist_step2)
        if indicator_step2 == '2SFD':
            success_prob = off_team_data['FTP'].iloc[0] / 100
            point = 0
            last_shot_success = True
            for _ in range(2):
                random_value = np.random.uniform(0, 1)
                if random_value <= success_prob:
                    point += 1
                    last_shot_success = True
                else:
                    last_shot_success = False

            if not last_shot_success:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = point

        elif indicator_step2 == '3SFD':
            success_prob = off_team_data['FTP'].iloc[0] / 100
            point = 0
            last_shot_success = True
            for _ in range(3):
                random_value = np.random.uniform(0, 1)
                if random_value <= success_prob:
                    point += 1
                    last_shot_success = True
                else:
                    last_shot_success = False

            if not last_shot_success:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = point

        else:
            success_prob = off_team_data['FTP'].iloc[0] / 100
            point = 0
            last_shot_success = True
            for _ in range(2):
                random_value = np.random.uniform(0, 1)
                if random_value <= success_prob:
                    point += 1
                    last_shot_success = True
                else:
                    last_shot_success = False

            if not last_shot_success:
                off_orb = off_team_data['ORB'].iloc[0]
                def_drb = def_team_data['DRB'].iloc[0]
                total = off_orb + def_drb
                prob_retry = off_orb / total if total > 0 else 0
                prob_fail = def_drb / total if total > 0 else 1

                retry_prob = np.random.uniform(0, 1)
                if retry_prob <= prob_retry:
                    retry_occurred = True
                    nested_result = simulation_function(OFF, DEF, rate, multiply, maindata)
                else:
                    point = point

    else:
        indicator_step2 = 'TOV'
        point = 0

    # 최종 결과 반환
    result = {
        'indicator_step1': indicator,
        'indicator_step2': indicator_step2,
        'point': point,
        'retry_occurred': retry_occurred
    }

    # 중첩 결과가 있는 경우 반영
    if nested_result is not None:
        result['nested_result'] = nested_result
        result['point'] += nested_result.get('point', 0)

    return result

In [15]:
simulation_function('ATL','BOS',0.8,100,Main_data)

{'indicator_step1': '2pt',
 'indicator_step2': 'RA',
 'point': 2,
 'retry_occurred': False}

### PACE

In [18]:
def game_simulation_function (OFF, DEF, rate, multiply, maindata):
    off_team_data = maindata[maindata['Team'] == OFF]
    def_team_data = maindata[maindata['Team'] == DEF]
    n=int((off_team_data['PACE'].iloc[0] + def_team_data['PACE'].iloc[0])/2)
    total_points = 0
    for _ in range(n):
        total_points += simulation_function(OFF, DEF, rate, multiply, maindata)['point']

    # 최종 누적 점수 반환
    return total_points

In [20]:
def schedule_simulation(schedule, rate, multiply, maindata):
    wins = []  # 승리 팀을 저장할 리스트
    loses = []  # 패배 팀을 저장할 리스트

    for _, row in tqdm(schedule.iterrows(), total=len(schedule), desc="Simulating Games"):
        visitor = row['Visitor']
        home = row['Home']

        # visitor가 OFF, home이 DEF일 때
        visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)

        # home이 OFF, visitor가 DEF일 때
        home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        # 무승부일 경우 한 번 더 시뮬레이션
        while visitor_off_points == home_off_points:
            visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)
            home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        # 승리/패배 팀 기록
        if visitor_off_points > home_off_points:
            wins.append(visitor)
            loses.append(home)
        else:
            wins.append(home)
            loses.append(visitor)

    # 결과를 schedule에 추가
    schedule['Win'] = wins
    schedule['Lose'] = loses
    return schedule


In [22]:
def schedule_simulation_basic(schedule, rate, multiply, maindata):
    wins = []  # 승리 팀을 저장할 리스트
    loses = []  # 패배 팀을 저장할 리스트

    for _, row in schedule.iterrows():
        visitor = row['Visitor']
        home = row['Home']

        # visitor가 OFF, home이 DEF일 때
        visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)

        # home이 OFF, visitor가 DEF일 때
        home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        # 무승부일 경우 한 번 더 시뮬레이션
        while visitor_off_points == home_off_points:
            visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)
            home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

        # 승리/패배 팀 기록
        if visitor_off_points > home_off_points:
            wins.append(visitor)
            loses.append(home)
        else:
            wins.append(home)
            loses.append(visitor)

    # 결과를 schedule에 추가
    schedule['Win'] = wins
    schedule['Lose'] = loses
    return schedule


In [24]:
def process_game(row, rate, multiply, maindata):
    visitor = row['Visitor']
    home = row['Home']

    # visitor가 OFF, home이 DEF일 때
    visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)

    # home이 OFF, visitor가 DEF일 때
    home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

    # 무승부일 경우 한 번 더 시뮬레이션
    while visitor_off_points == home_off_points:
        visitor_off_points = game_simulation_function(visitor, home, rate, multiply, maindata)
        home_off_points = game_simulation_function(home, visitor, rate, multiply, maindata)

    # 결과 반환
    if visitor_off_points > home_off_points:
        return visitor, home
    else:
        return home, visitor

def schedule_simulation_parallel(schedule, rate, multiply, maindata, n_jobs=-1):
    # 병렬 처리로 게임 결과 계산
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_game)(row, rate, multiply, maindata) for _, row in tqdm(schedule.iterrows(), total=len(schedule), desc="Simulating Games")
    )

    # 결과를 schedule에 추가
    wins, loses = zip(*results)
    schedule['Win'] = wins
    schedule['Lose'] = loses
    return schedule


In [26]:
def schedule_simulation_ntimes(n, schedule, rate, multiply, maindata):
    expanded_schedules = []  # 모든 시뮬레이션 결과를 저장할 리스트

    for i in range(n):
        # tqdm을 사용하여 진행 상태를 표시
        print(f"Simulation {i + 1}/{n}")
        simulated_schedule = schedule_simulation_parallel(schedule.copy(), rate, multiply, maindata)
        simulated_schedule['Simulation'] = i + 1  # 반복 횟수를 기록
        expanded_schedules.append(simulated_schedule)

    # 반복 결과를 하나의 데이터프레임으로 합침
    combined_schedule = pd.concat(expanded_schedules, ignore_index=True)
    return combined_schedule


In [28]:
def calculate_win_percentage(schedule, main_data):
    import pandas as pd

    # 승리(Wins)와 패배(Losses) 횟수 집계
    win_counts = schedule['Win'].value_counts()  # 팀별 승리 횟수
    lose_counts = schedule['Lose'].value_counts()  # 팀별 패배 횟수

    # 총 경기 횟수 계산
    game_counts = win_counts.add(lose_counts, fill_value=0)  # 승리 + 패배 (NaN 값은 0으로 처리)

    # 승률 계산
    win_percentage = (win_counts / game_counts).fillna(0)  # 승률 = 승리 횟수 / 총 경기 횟수

    # 데이터프레임으로 정리
    win_percentage_table = pd.DataFrame({
        'Team': win_counts.index.union(lose_counts.index),  # 모든 팀 포함
        'Conference': win_counts.index.union(lose_counts.index).map(main_data.set_index('Team')['Conference']),  # Conference 추가
        'Wins': win_counts,  # 승리 횟수
        'Losses': lose_counts,  # 패배 횟수
        'Win_Percentage': win_percentage  # 승률
    }).fillna(0)  # NaN 값을 0으로 처리

    # 정수형으로 변환
    win_percentage_table['Wins'] = win_percentage_table['Wins'].astype(int)
    win_percentage_table['Losses'] = win_percentage_table['Losses'].astype(int)

    # 각 Conference별로 상위 10팀 선별
    win_percentage_table['Play_off'] = 0  # 기본값 0으로 설정
    win_percentage_table.loc[win_percentage_table.groupby('Conference')['Win_Percentage']
                             .rank(method='first', ascending=False) <= 10, 'Play_off'] = 1

    # 인덱스 재설정
    win_percentage_table = win_percentage_table.reset_index(drop=True)

    return win_percentage_table


In [30]:
from sklearn.metrics import mean_absolute_error

def calculate_mae(result_sample, origin_result):
    # result_sample이 Series라면 DataFrame으로 변환
    if isinstance(result_sample, pd.Series):
        result_sample = result_sample.reset_index()
        result_sample.columns = ['Team', 'Win_Percentage']

    # origin_result도 마찬가지로 확인
    if isinstance(origin_result, pd.Series):
        origin_result = origin_result.reset_index()
        origin_result.columns = ['Team', 'Win_Percentage']

    # 데이터프레임 병합
    comparison_df = result_sample.merge(origin_result, on='Team', suffixes=('_Sample', '_Origin'))

    # MAE 계산
    mae = mean_absolute_error(
        comparison_df['Win_Percentage_Sample'], 
        comparison_df['Win_Percentage_Origin']
    )
    return mae


In [32]:
def calculate_play_off_accuracy(predicted_result, origin_result):
    import pandas as pd

    # Play_off 비교
    merged = predicted_result[['Team', 'Play_off']].merge(origin_result[['Team', 'Play_off']], on='Team', suffixes=('_predicted', '_actual'))

    # Accuracy 계산
    accuracy = (merged['Play_off_predicted'] == merged['Play_off_actual']).mean()

    return accuracy


In [39]:
import pandas as pd

team_comparison_list = []
results_acc = []
results_mae = []

for i in range(50):
    print(f"Running simulation {i+1}")
    # 시뮬레이션 실행
    exam_n = schedule_simulation_ntimes(1, Schedule, 0.98, 67, Main_data)

    # Win_Percentage 및 Play_off 계산
    sample = calculate_win_percentage(exam_n, Main_data)
    accuracy = calculate_play_off_accuracy(sample, Origin_result)
    mae = calculate_mae(sample, Origin_result)

    # sample과 Origin_result를 병합하여 비교
    comparison = sample[['Team', 'Win_Percentage', 'Play_off']].merge(
        Origin_result[['Team', 'Win_Percentage', 'Play_off']], 
        on='Team', 
        suffixes=('_sim', '_origin')
    )
    
    # 팀별 MAE 계산 및 플레이오프 정확도 확인
    comparison['MAE'] = abs(comparison['Win_Percentage_sim'] - comparison['Win_Percentage_origin'])
    comparison['Play_off_Correct'] = (comparison['Play_off_sim'] == comparison['Play_off_origin']).astype(int)

    # 시뮬레이션 번호 추가
    comparison['Season'] = i + 1  # 1부터 시작하는 시즌 번호 추가

    # 결과 저장
    team_comparison_list.append(comparison)
    results_acc.append(accuracy)
    results_mae.append(mae)

# 모든 반복 결과를 하나의 데이터프레임으로 병합 (시즌별 기록 포함)
combined_comparison = pd.concat(team_comparison_list)

# 시즌별로 그룹화하여 분석 가능
season_results = combined_comparison.groupby(['Season', 'Team']).agg({
    'Win_Percentage_sim': 'mean',
    'Play_off_sim': 'mean',
    'MAE': 'mean'
}).reset_index()

# 팀별 MAE와 플레이오프 예측 정확도 평균 및 표준편차 계산
team_results = combined_comparison.groupby('Team').agg({
    'MAE': ['mean', 'std'],  # MAE 평균 및 표준편차
    'Play_off_Correct': ['mean', 'std']  # 플레이오프 예측 정확도의 평균 (비율) 및 표준편차
}).reset_index()

# 컬럼명 정리
team_results.columns = ['Team', 'MAE_mean', 'MAE_std', 'Play_off_Accuracy (%)', 'Play_off_Correct_std']
team_results['Play_off_Accuracy (%)'] *= 100  # 정확도를 %로 변환
team_results['Play_off_Correct_std'] *= 100  # 표준편차도 %로 변환

# 전체 평균 및 표준편차 계산
average_acc = sum(results_acc) / len(results_acc)
average_mae = sum(results_mae) / len(results_mae)
std_acc = pd.Series(results_acc).std()
std_mae = pd.Series(results_mae).std()

# 시즌별 결과를 xlsx 파일로 저장
season_results.to_excel("season_results.xlsx", index=False)

print("Season results have been saved as 'season_results.xlsx'.")

# 최종 결과 출력
print(f"Average Play-off Accuracy: {average_acc:.4f} ± {std_acc:.4f}")
print(f"Average MAE: {average_mae:.4f} ± {std_mae:.4f}")

# 시즌별 결과 테이블 출력
print("\nSeasonal Results:")
print(season_results.to_string(index=False))

# 팀별 평균 결과 테이블 출력
print("\nTeam Results:")
print(team_results.to_string(index=False))


Running simulation 1
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Running simulation 2
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Running simulation 3
Simulation 1/1


...

Running simulation 49
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Running simulation 50
Simulation 1/1


Simulating Games:   0%|          | 0/1230 [00:00<?, ?it/s]

Season results have been saved as 'season_results.xlsx'.
Average Play-off Accuracy: 0.8360 ± 0.0575
Average MAE: 0.0897 ± 0.0084

Seasonal Results:
 Season Team  Win_Percentage_sim  Play_off_sim      MAE
      1  ATL            0.536585           1.0 0.097585
      1  BKN            0.390244           0.0 0.000244
      1  BOS            0.634146           1.0 0.155854
      1  CHA            0.390244           0.0 0.134244
      1  CHI            0.487805           1.0 0.011805
      1  CLE            0.463415           0.0 0.121585
      1  DAL            0.512195           1.0 0.097805
      1  DEN            0.634146           1.0 0.060854
      1  DET            0.500000           1.0 0.329000
      1  GSW            0.658537           1.0 0.097537
      1  HOU            0.548780           1.0 0.048780
      1  IND            0.597561           1.0 0.024561
      1  LAC            0.512195           1.0 0.109805
      1  LAL            0.560976           1.0 0.012024
      1  MEM

### 쓰레기통